In [1]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde molecule in a cc-pVDZ basis set.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from helper_cis import *
from matplotlib import pyplot as plt
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [13]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
print(energy)

    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=5, triplets = "NONE", tda=True)
#print(res)

# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

psi4_tdm = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]

#print(psi4_excitation_e)
#print(psi4_tdm)

# run helper_cis()
cis_e, cis_c, cis_mu, tdm = cis(molstr, options_dict)
print("psi4 dE")
print(psi4_excitation_e[0] * 27.211, psi4_excitation_e[1] * 27.211, psi4_excitation_e[2] * 27.211, psi4_excitation_e[3] * 27.211, psi4_excitation_e[4] * 27.211)
print("my dE")
print(cis_e[0:5]*27.211)

print(psi4_tdm)
#[0.1662481187552202, 0.3638308461069341, 0.4749211707441484, 0.5118092417869102, 0.6846051041776717, 0.7239895336225287, 0.7855788521719436, 0.7947894069887481, 0.7949568461484611, 0.8106250052881367]

-113.87722217661789
printing eps
[-20.57446798 -11.3410548   -1.41949721  -0.86260316  -0.69345807
  -0.65299446  -0.53953067  -0.43218362   0.14321496   0.19105205
   0.2605678    0.36232178   0.66779436   0.6730188    0.74021896
   0.81939072   0.91783391   0.98277694   1.10277687   1.15795498
   1.28991918   1.32357041   1.42213733   1.67659278   1.69713706
   1.86383505   1.95694968   1.97335868   1.98243242   2.00161686
   2.5881272    2.68978681   2.75895549   3.05184146   3.19168357
   3.44674875   3.75149892   3.8698512 ]

Number of occupied orbitals: 8
psi4 dE
4.726755504001224 10.286294990651296 10.35053938898499 10.556573151383908 11.907679441387744
my dE
[ 4.72675547 10.28629496 10.35053933 10.55657315 11.90767941]
[array([ 1.15930044e-15, -3.06690717e-16, -3.07492898e-16]), array([ 4.57978222e-02,  2.94977812e-14, -1.69465683e-15]), array([ 1.20416738e-15, -9.95224321e-01, -2.93178642e-13]), array([-5.70838140e-17,  5.86910855e-13, -7.98961702e-01]), array([-4.26152588e-15

In [17]:
# number of doubly occupied orbitals
ndocc   = wfn.nalpha()
    
# total number of orbitals
nmo     = wfn.nmo()
    
# number of virtual orbitals
nvirt   = nmo - ndocc
for i in range(0, ndocc):
    for a in range(0, nvirt):
        ia = i*nvirt + a
        if cis_c[ia,3]*np.conj(cis_c[ia,3]) > 1e-3:
            print(i+1,"->",a+1+ndocc,cis_c[ia,3]*np.conj(cis_c[ia,3]))


3 -> 12 0.001081777688032667
4 -> 10 0.004209285548142989
5 -> 11 0.011590508930605911
5 -> 16 0.0022861342672535335
5 -> 22 0.0012545750238114168
6 -> 12 0.005505181493515919
6 -> 17 0.007321367522312326
6 -> 18 0.006350589572080172
7 -> 9 0.8357843341635882
7 -> 13 0.0027169243826936284
8 -> 11 0.09013909120473372
8 -> 15 0.015543032945466693
8 -> 16 0.005439889084101332
8 -> 22 0.0016693312041454605
8 -> 28 0.0014156427172183138


In [44]:
# omega ~resonant with the S0->S3 and S0->S4 transitions that have tdm along the x- and y-axis, respectively
om = 10.4/27.211

# different magnitudes of the lambda vector
l = 0.2

no_lam = np.array([0., 0., 0.])
lam_y = np.array([0, l, 0])
lam_z = np.array([0, 0, l])
lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])
    
# results for y-polarized
y_dict = cs_cqed_cis(lam_y, om, molstr, options_dict)
    
z_dict = cs_cqed_cis(lam_z, om, molstr, options_dict)

yz_dict = cs_cqed_cis(lam_yz, om, molstr, options_dict)




C
38
Cocc
38

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999194131
CQED-RHF One-electron energy      = -217.9897365386672448
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -113.7363030943151898   dE = -1.13736E+02   dRMS = 2.04063E-03
SCF Iteration   2: Energy = -113.7414221172238342   dE = -5.11902E-03   dRMS = 6.67606E-04
SCF Iteration   3: Energy = -113.7417301257044357   dE = -3.08008E-04   dRMS = 3.16945E-04
SCF Iteration   4: Energy = -113.7417743475720044   dE = -4.42219E-05   dRMS = 2.21700E-04
SCF Iteration   5: Energy = -113.7417876864722928   dE = -1.33389E-05   dRMS = 1.66260E-04
SCF Iteration   6: Energy = -113.7417939719871924   dE = -6.28551E-06   dRMS = 1.23578E-04
SCF Iteration   7: Energy = -113.7417973912125433   dE = -3.41923E-06   dRMS = 9.48446E-05
SCF Iteration   8: Energy = -113.7417993297345333   dE = -1.93852E-06   dRMS = 7.11153E-05
SCF Iter

SCF Iteration  38: Energy = -113.7155969999141405   dE =  2.98428E-13   dRMS = 3.39245E-09
SCF Iteration  39: Energy = -113.7155969999144531   dE = -3.12639E-13   dRMS = 2.40897E-09
SCF Iteration  40: Energy = -113.7155969999140837   dE =  3.69482E-13   dRMS = 1.71143E-09
SCF Iteration  41: Energy = -113.7155969999142826   dE = -1.98952E-13   dRMS = 1.21538E-09
SCF Iteration  42: Energy = -113.7155969999144247   dE = -1.42109E-13   dRMS = 8.63397E-10
SCF Iteration  43: Energy = -113.7155969999144105   dE =  1.42109E-14   dRMS = 6.13176E-10
SCF Iteration  44: Energy = -113.7155969999142684   dE =  1.42109E-13   dRMS = 4.35578E-10
SCF Iteration  45: Energy = -113.7155969999142826   dE = -1.42109E-14   dRMS = 3.09355E-10
SCF Iteration  46: Energy = -113.7155969999142542   dE =  2.84217E-14   dRMS = 2.19747E-10
SCF Iteration  47: Energy = -113.7155969999143537   dE = -9.94760E-14   dRMS = 1.56072E-10
SCF Iteration  48: Energy = -113.7155969999143394   dE =  1.42109E-14   dRMS = 1.10862E-10

In [45]:
no_lam_no_om_dict = cs_cqed_cis(no_lam, 0, molstr, options_dict)
no_lam_dict = cs_cqed_cis(no_lam, om, molstr, options_dict)


C
38
Cocc
38

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999194131
CQED-RHF One-electron energy      = -218.2835959999194131
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -113.8772221766176642   dE = -1.13877E+02   dRMS = 6.35492E-12
SCF Iteration   2: Energy = -113.8772221766179769   dE = -3.12639E-13   dRMS = 3.06563E-12
Total time for SCF iterations: 0.023 seconds 

QED-RHF   energy: -113.87722218 hartree
Psi4  SCF energy: -113.87722218 hartree
C
38
Cocc
38

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999194131
CQED-RHF One-electron energy      = -218.2835959999194131
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -113.8772221766176642   dE = -1.13877E+02   dRMS = 6.35492E-12
SCF Iteration   2: Energy = -113.8772221766179769   dE = -3.12639E-13   

In [46]:
print("No lambda No Omega")
print(no_lam_no_om_dict['CQED-CIS ENERGY'][:10]*27.211)

print("No lambda")
print(no_lam_dict['CQED-CIS ENERGY'][:10]*27.211)

print("lambda_y")
print(y_dict['CQED-CIS ENERGY'][:10]*27.211)

print("lambda_z")
print(z_dict['CQED-CIS ENERGY'][:10]*27.211)

print("lambda_yz")
print(yz_dict['CQED-CIS ENERGY'][:10]*27.211)

No lambda No Omega
[ 0.          0.          4.72675547  4.72675547 10.28629496 10.28629496
 10.35053933 10.35053933 10.55657315 10.55657315]
No lambda
[ 0.          4.72675547 10.28629496 10.35053933 10.4        10.55657315
 11.90767941 12.87530853 12.88675301 13.36236287]
lambda_y
[-0.70728552  4.73527355  8.45937597 10.25444198 10.72659913 11.99492213
 12.47981242 13.28189138 13.62406809 14.15412051]
lambda_z
[-1.04614171  2.03027975  7.01742712  7.52843383  8.44543768  8.94162702
 11.13404731 11.17349136 11.19571266 12.37016226]
lambda_yz
[-0.87199387  3.2202727   7.80477056  8.64991763  9.4653857  10.48402973
 11.31790942 12.05964974 12.58322794 13.49516881]


In [37]:
no_lam_no_om_c = no_lam_no_om_dict['CQED-CIS L VECTORS']
no_lam_c = no_lam_dict['CQED-CIS L VECTORS']
lam_y_c = y_dict['CQED-CIS L VECTORS']
lam_z_c = z_dict['CQED-CIS L VECTORS']
lam_yz_c = yz_dict['CQED-CIS L VECTORS']



state = 0
print(" excitation with no lambda no omega ground state")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = no_lam_no_om_c[ia0,state] * np.conj(no_lam_no_om_c[ia0,state])
        amp01 = no_lam_no_om_c[ia1,state] * np.conj(no_lam_no_om_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00)
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01)
            
            
print(" excitation with no lambda ground state")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = no_lam_c[ia0,state] * np.conj(no_lam_c[ia0,state])
        amp01 = no_lam_c[ia1,state] * np.conj(no_lam_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00)
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01)
            
print(" excitation with lambda_y state")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_y_c[ia0,state] * np.conj(lam_y_c[ia0,state])
        amp01 = lam_y_c[ia1,state] * np.conj(lam_y_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00)
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01)
            

print(" excitation with lambda_z state")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_z_c[ia0,state] * np.conj(lam_z_c[ia0,state])
        amp01 = lam_z_c[ia1,state] * np.conj(lam_z_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00)
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01)
            
print(" excitation with lambda_yz state")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_yz_c[ia0,state] * np.conj(lam_yz_c[ia0,state])
        amp01 = lam_yz_c[ia1,state] * np.conj(lam_yz_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00)
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01)
            
   

 excitation with no lambda no omega ground state
 excitation with no lambda ground state
 excitation with lambda_y state
5 ;0-> 10 ;1 (0.0036808329408522084+0j)
5 ;0-> 11 ;1 (0.0024254993494503157+0j)
6 ;0-> 12 ;1 (0.0014234998098610454+0j)
8 ;0-> 10 ;1 (0.005458122446978971+0j)
8 ;0-> 14 ;1 (0.001095429844030031+0j)
 excitation with lambda_z state
4 ;0-> 12 ;1 (0.0012518927055941908+0j)
6 ;0-> 12 ;1 (0.0022482959088339456+0j)
7 ;0-> 9 ;0 (0.002919669914392131+0j)
7 ;0-> 9 ;1 (0.014922685056668421+0j)
8 ;0-> 11 ;1 (0.004886271564006125+0j)
 excitation with lambda_yz state
4 ;0-> 12 ;1 (0.0018402434390781789+0j)
5 ;0-> 11 ;1 (0.001079012280672516+0j)
5 ;0-> 12 ;1 (0.0015706687442300989+0j)
6 ;0-> 12 ;1 (0.0012075036905632142+0j)
7 ;0-> 9 ;1 (0.007276699487663493+0j)
8 ;0-> 12 ;1 (0.004365229223147301+0j)
